# Registration and Demographic Data DVA for Personal Agendas

## Necesary Files
Refresed every Week

- registration_BVA24_BVA25.json
- registration_LVS24.json
- demographics_BVA24_BVA25.json
- demographics_LVS24.json

## list of QA to keep 

- list_keep_this = ['What type does your practice specialise in', 'Organisation Type','Job Role']
- list_keep_past = [ 'What areas do you specialise in', 'Organisation Type','Job Role']

In [ ]:
import pandas as pd
import json
from pandas import json_normalize
import warnings
from typing import List
from pandas.errors import SettingWithCopyWarning
import logging

warnings.simplefilter(action="ignore", category=(SettingWithCopyWarning))

In [ ]:
with open(
    "data/bva/20250428_registration_BVA24_BVA25.json", "r", encoding="utf-8"
) as f:  # 'csm_data/BVA/20250305_reg_visitor_BVA24_BVA25.json
    regdata = json.load(f)

In [ ]:
with open(
    "data/bva/20250428_registration_LVS24.json", "r", encoding="utf-8"
) as f:  # 'csm_data/BVA/20250305_reg_visitor_BVA24_BVA25.json
    regdata_lva = json.load(f)

In [ ]:
columns = list(regdata[0].keys())
columns

In [ ]:
columns = list(regdata_lva[0].keys())
columns

In [ ]:
# Normalize the nested JSON
df = json_normalize(regdata)
df_lva = json_normalize(regdata_lva)

In [ ]:
df.to_csv("data/bva/csv/Registration_data_bva.csv", index=False)
df_lva.to_csv("data/bva/csv/Registration_data_lva.csv", index=False)

In [ ]:
len(df), len(
    df_lva
)  # 08042025 Before = 3092 # 16042025--> (4772, 11426) # Before 22042025 -->(4831, 11426) # before 29042025 (4913, 11426)

# JOB title or Job Role from Demographic

In [ ]:
df["JobTitle"] = df["JobTitle"].str.lower()
df["JobTitle"] = df["JobTitle"].fillna("NA")

In [ ]:
df.head()

# Preprocessing Registration Data

In [ ]:
from datetime import datetime
import string


def calculate_date_difference(df, date_column, given_date_str):
    # Convert the column to datetime
    df[date_column] = pd.to_datetime(df[date_column])

    # Convert the given date to datetime
    given_date = datetime.strptime(given_date_str, "%Y-%m-%d")

    # Calculate the difference in days
    df["Days_since_registration"] = (given_date - df[date_column]).dt.days

    return df

In [ ]:
# Calculate the difference in days from a given date
given_date_str_this = "2025-06-12"  # Event Day
given_date_str_last = "2024-06-12"  # Event Day
# result_df = calculate_date_difference(df, 'RegistrationDate', given_date_str)

In [ ]:
df = df.dropna(subset=["Email"])
df = df.dropna(subset=["Forename"])
df = df.dropna(subset=["Surname"])

In [ ]:
df_lva = df_lva.dropna(subset=["Email"])
df_lva = df_lva.dropna(subset=["Forename"])
df_lva = df_lva.dropna(subset=["Surname"])

In [ ]:
df.isnull().sum()

In [ ]:
list_of_professions = list(df["JobTitle"].unique())

In [ ]:
job_titles = pd.DataFrame(data=list_of_professions, columns=["JobTitle"])
job_titles.to_csv("data/bva/list_of_professions.csv", index=False)

In [ ]:
len(list_of_professions), len(df)  # 22042025 (1354, 4821)

In [ ]:
df["ShowRef"].unique()

In [ ]:
df["Company"].unique()

In [ ]:
df_lva["ShowRef"].unique()

In [ ]:
df["BadgeType"].unique()

In [ ]:
df_lva["BadgeType"].unique()

In [ ]:
df["Source"].unique()

In [ ]:
df["Status"].unique()

In [ ]:
shows_25 = ["BVA2025"]

In [ ]:
df["RegCode"].unique()

In [ ]:
# Registro 24 and 25
df_reg_25 = df[df.ShowRef.isin(shows_25)]
df_reg_24 = df[~(df.ShowRef.isin(shows_25))]
len(df_reg_25), len(df_reg_24)

In [ ]:
df_reg_25["BadgeType"].unique()

In [ ]:
len(df_lva[df_lva["BadgeType"] == "Delegate - Group"]), len(
    df_reg_24[df_reg_24["BadgeType"] == "Delegate - Group"]
), len(df_reg_25[df_reg_25["BadgeType"] == "Delegate - Group"])

In [ ]:
df["BadgeType"].unique()

# Drop duplicates badgeID and remove non-valid BadgeTypes

In [ ]:
list_valid_badges_types = ["Delegate", "Delegate - Group"]

In [ ]:
df_reg_25 = df_reg_25[df_reg_25["BadgeType"].isin(list_valid_badges_types)]
df_reg_24 = df_reg_24[df_reg_24["BadgeType"].isin(list_valid_badges_types)]
df_lva_24 = df_lva[df_lva["BadgeType"].isin(list_valid_badges_types)]

len(df_reg_25), len(df_reg_24), len(df_lva_24)

In [ ]:
len(df_reg_25["BadgeId"].unique()), len(df_reg_24["BadgeId"].unique()), len(
    df_lva_24["BadgeId"].unique()
)

In [ ]:
df_reg_25 = df_reg_25.drop_duplicates(subset="BadgeId", keep="first")
df_reg_24 = df_reg_24.drop_duplicates(subset="BadgeId", keep="first")
df_lva_24 = df_lva_24.drop_duplicates(subset="BadgeId", keep="first")
len(df_reg_25), len(df_reg_24), len(df_lva_24)

In [ ]:
# save registration data
df_reg_25.to_csv("data/bva/csv/Registration_data_bva_25_raw.csv", index=False)
df_reg_24.to_csv("data/bva/csv/Registration_data_bva_24_raw.csv", index=False)
df_lva_24.to_csv("data/bva/csv/Registration_data_lva_24_raw.csv", index=False)

In [ ]:
def concat_forename_email(row):
    return (
        row["Forename"].lower()
        + "_"
        + row["Surname"].lower()
        + "_"
        + row["Email"].lower()
    )

In [ ]:
df_reg_24["id_both_years"] = df_reg_24.apply(
    lambda row: concat_forename_email(row), axis=1
)
df_reg_25["id_both_years"] = df_reg_25.apply(
    lambda row: concat_forename_email(row), axis=1
)
df_lva_24["id_both_years"] = df_lva_24.apply(
    lambda row: concat_forename_email(row), axis=1
)

In [ ]:
df_reg_25.head(2)

In [ ]:
# list email 24 and 25
list_email_24 = set(list(df_reg_24["id_both_years"].unique()))
list_email_25 = set(list(df_reg_25["id_both_years"].unique()))
list_lva_email_24 = set(list(df_lva_24["id_both_years"].unique()))
visitors_before_this_year_bva = list_email_25.intersection(list_email_24)
visitors_before_this_year_lva = list_email_25.intersection(list_lva_email_24)
len(visitors_before_this_year_bva), len(visitors_before_this_year_lva)

In [ ]:
len(list_email_24), len(list_lva_email_24)

In [ ]:
# registration Data users that were in 2024 and 2025
df_reg_24_25_bva = df_reg_24[
    df_reg_24["id_both_years"].isin(visitors_before_this_year_bva)
]
df_reg_24_25_lva = df_lva_24[
    df_lva_24["id_both_years"].isin(visitors_before_this_year_lva)
]

In [ ]:
df_reg_25 = calculate_date_difference(
    df_reg_25, "RegistrationDate", given_date_str_this
)
df_reg_24 = calculate_date_difference(
    df_reg_24, "RegistrationDate", given_date_str_this
)

df_reg_24_25_bva = calculate_date_difference(
    df_reg_24_25_bva, "RegistrationDate", given_date_str_last
)
df_reg_24_25_lva = calculate_date_difference(
    df_reg_24_25_lva, "RegistrationDate", given_date_str_last
)
df_reg_25.head(2)

# Get domain and flag if assist last year event and BadID_Last_year

In [ ]:
badge_last_year_index_bva = df_reg_24_25_bva[["BadgeId", "id_both_years"]]
badge_last_year_index_lva = df_reg_24_25_lva[["BadgeId", "id_both_years"]]

In [ ]:
badge_last_year_index_bva.columns = ["BadgeId_last_year_bva", "id_both_years"]
badge_last_year_index_lva.columns = ["BadgeId_last_year_lva", "id_both_years"]

In [ ]:
df_reg_25 = pd.merge(
    df_reg_25, badge_last_year_index_bva, on=["id_both_years"], how="left"
)
df_reg_25 = pd.merge(
    df_reg_25, badge_last_year_index_lva, on=["id_both_years"], how="left"
)

In [ ]:
df_reg_25["BadgeId_last_year_bva"] = df_reg_25["BadgeId_last_year_bva"].fillna("NA")
df_reg_25["BadgeId_last_year_lva"] = df_reg_25["BadgeId_last_year_lva"].fillna("NA")

In [ ]:
len(df_reg_25["BadgeId_last_year_bva"].unique()), len(
    df_reg_25["BadgeId_last_year_lva"].unique()
)  # must be 1 more than len(visitors_before_this_year)

In [ ]:
def return_domain(x):
    return x.split("@")[-1]


def assits_year_before(x, list_email):
    if x["id_both_years"] in list_email:
        return 1
    else:
        return 0

# UNion of visitor that have been in BVA and LVA past year that repeat this year

In [ ]:
visitors_before_this_year = visitors_before_this_year_bva.union(
    visitors_before_this_year_lva
)
len(visitors_before_this_year)

In [ ]:
df_reg_25["Email_domain"] = df_reg_25["Email"].apply(return_domain)
df_reg_25["assist_year_before"] = df_reg_25.apply(
    lambda row: assits_year_before(row, visitors_before_this_year), axis=1
)

# last year
df_reg_24["Email_domain"] = df_reg_24["Email"].apply(return_domain)
df_reg_24_25_bva["Email_domain"] = df_reg_24_25_bva["Email"].apply(return_domain)
df_reg_24_25_lva["Email_domain"] = df_reg_24_25_lva["Email"].apply(return_domain)

In [ ]:
df_reg_25["assist_year_before"].value_counts()

In [ ]:
len(df_reg_25["BadgeId"].unique()), len(df_reg_24_25_bva["BadgeId"].unique()), len(
    df_reg_24_25_lva["BadgeId"].unique()
)

In [ ]:
 "GN22YN9" in list(df_reg_24['BadgeId'].unique())

In [ ]:
# save registration data
df_reg_25.to_csv("data/bva/csv/Registration_data_bva_25.csv", index=False)
df_reg_24.to_csv("data/bva/csv/Registration_data_bva_24.csv", index=False)
df_reg_24_25_bva.to_csv("data/bva/csv/Registration_data_bva_24_25.csv", index=False)
df_reg_24_25_lva.to_csv("data/bva/csv/Registration_data_bva_24_25.csv", index=False)

In [ ]:
df_reg_25.columns

In [ ]:
# Select Valid Columns
columns_this = [
    "Email",
    "Email_domain",
    "Company",
    "JobTitle",
    "Country",
    "BadgeType",
    "ShowRef",
    "BadgeId",
    "Source",
    "Days_since_registration",
    "assist_year_before",
    "BadgeId_last_year_bva",
    "BadgeId_last_year_lva",
]
columns_last = [
    "Email",
    "Email_domain",
    "Company",
    "JobTitle",
    "Country",
    "BadgeType",
    "ShowRef",
    "BadgeId",
    "Source",
    "Days_since_registration",
]

In [ ]:
df_reg_25_valid_columns = df_reg_25[columns_this]
df_reg_24_valid_columns = df_reg_24[columns_last]
df_reg_24_25_bva_valid_columns = df_reg_24_25_bva[columns_last]
df_reg_24_25_lva_valid_columns = df_reg_24_25_lva[columns_last]

In [ ]:
df_reg_25_valid_columns.isnull().sum()

In [ ]:
df_reg_25_valid_columns.head()

In [ ]:
df_reg_24_valid_columns.isnull().sum()

In [ ]:
# Save data
df_reg_25_valid_columns.to_csv(
    "data/bva/csv/Registration_data_bva_25_only_valid.csv", index=False
)
df_reg_24_valid_columns.to_csv(
    "data/bva/csv/Registration_data_bva_24_only_valid.csv", index=False
)
df_reg_24_25_bva_valid_columns.to_csv(
    "data/bva/csv/Registration_data_bva_24_25_only_valid.csv", index=False
)
df_reg_24_25_lva_valid_columns.to_csv(
    "data/bva/csv/Registration_data_lva_24_25_only_valid.csv", index=False
)

In [ ]:
# AQUI

# Demographic Data BVA and LVA

In [ ]:
with open("data/bva/20250428_demographics_BVA24_BVA25.json", "r") as f:
    demodata = json.load(f)

In [ ]:
# Normalize the nested JSON
df2 = json_normalize(demodata)

In [ ]:
with open("data/bva/20250428_demographics_LVS24.json", "r") as f:
    demodata_lva = json.load(f)

In [ ]:
# Normalize the nested JSON
df3 = json_normalize(demodata_lva)

In [ ]:
len(df2), len(
    demodata_lva
)  # 080425 before 25269 # 16042025 (31700, 65186) # 22042025 (32131, 65186) #29042025 (33395, 65186)

In [ ]:
df2.to_csv("data/bva/csv/Registration_demographicdata_bva.csv", index=False)
df3.to_csv("data/bva/csv/Registration_demographicdata_lva.csv", index=False)

In [ ]:
df_demo_25 = df2[df2.showref.isin(shows_25)]
df_demo_24 = df2[~(df2.showref.isin(shows_25))]
len(df_demo_25), len(df_demo_24)

In [ ]:
def remove_punctuation(text):
    """Removes punctuation from a string, excluding hyphens."""
    custom_punctuation = (
        string.punctuation.replace("-", "").replace("/", "") + "‘’“”…" + "â€™Â"
    )
    return text.translate(str.maketrans("", "", custom_punctuation))

In [ ]:
df3.head()

In [ ]:
df_demo_25["QuestionText"] = df_demo_25["QuestionText"].apply(remove_punctuation)
df_demo_24["QuestionText"] = df_demo_24["QuestionText"].apply(remove_punctuation)
df3["QuestionText"] = df3["QuestionText"].apply(remove_punctuation)

In [ ]:
job_records_df = df2[df2["QuestionText"] == "Job Role"]
job_records_df.head(3)

In [ ]:
job_records_df_lva = df3[df3["QuestionText"] == "Job Role"]
job_records_df_lva.head(3)

# Spezialization

In [ ]:
df_spe = df_demo_25[
    df_demo_25["QuestionText"] == "What type does your practice specialise in"
]
df_spe2 = df_demo_24[df_demo_24["QuestionText"] == "What areas do you specialise in"]

In [ ]:
original_list = df_spe["AnswerText"].unique()

In [ ]:
# Set to store unique classes
unique_classes = set()

# Process each item in the list
for item in original_list:
    # Split by semicolon and add each class to the set
    classes = item.split(";")
    for cls in classes:
        unique_classes.add(cls.strip())

# Convert set back to a list for the final result
individual_classes = list(unique_classes)

print("Number of unique classes:", len(individual_classes))
print("Unique classes:", individual_classes)

In [ ]:
df_spe2["AnswerText"].unique()

In [ ]:
df_spe3 = df3[df3["QuestionText"] == "What areas do you specialise in"]

In [ ]:
original_list2 = df_spe3["AnswerText"].unique()
# Set to store unique classes
unique_classes = set()

# Process each item in the list
for item in original_list2:
    # Split by semicolon and add each class to the set
    classes = item.split(";")
    for cls in classes:
        unique_classes.add(cls.strip())

# Convert set back to a list for the final result
individual_classes = list(unique_classes)

print("Number of unique classes:", len(individual_classes))
print("Unique classes:", individual_classes)

In [ ]:
# Jobsroles

In [ ]:
job_records_df.isnull().sum()

In [ ]:
job1 = set(list(job_records_df["AnswerText"].unique()))

In [ ]:
job2 = set(list(job_records_df_lva["AnswerText"].unique()))

In [ ]:
job1.union(job2)

In [ ]:
df_demo_25.to_csv(
    "data/bva/csv/Registration_demographicdata_bva_25_raw.csv", index=False
)
df_demo_24.to_csv(
    "data/bva/csv/Registration_demographicdata_bva_24_raw.csv", index=False
)
df3.to_csv("data/bva/csv/Registration_demographicdata_lva_24_raw.csv", index=False)

In [ ]:
# lists of QA
list_25 = set(list(df_demo_25["QuestionText"].unique()))
list_24_bva = set(list(df_demo_24["QuestionText"].unique()))
list_24_lva = set(list(df3["QuestionText"].unique()))
to_exclude = set(["UTMSOURCE", "UTMMEDIUM", "UTMCAMPAIGN", "TRACESOURCECODE"])
list_25 = list_25 - to_exclude
common_qa_bva = list_25.intersection(list_24_bva)
common_qa_lva = list_25.intersection(list_24_lva)

In [ ]:
# list_24

In [ ]:
list_25

In [ ]:
# common_qa_lva

# QA of users in 2024 that repeat in 2025

In [ ]:
list_badgeid_24_25_bva = set(list(df_reg_24_25_bva["BadgeId"].unique()))
list_badgeid_24_25_lva = set(list(df_reg_24_25_lva["BadgeId"].unique()))
len(list_badgeid_24_25_bva), len(list_badgeid_24_25_lva)

In [ ]:
df_demo_24_25_bva = df_demo_24[df_demo_24["BadgeId"].isin(list_badgeid_24_25_bva)]
df_demo_24_25_lva = df3[df3["BadgeId"].isin(list_badgeid_24_25_lva)]
len(df_demo_24_25_bva), len(df_demo_24_25_lva)

In [ ]:
df_demo_25.to_csv("data/bva/csv/Registration_demographicdata_bva_25.csv", index=False)
df_demo_24.to_csv("data/bva/csv/Registration_demographicdata_bva_24.csv", index=False)
df3.to_csv("data/bva/csv/Registration_demographicdata_lva_24.csv", index=False)
df_demo_24_25_bva.to_csv(
    "data/bva/csv/Registration_demographicdata_bva_24_25.csv", index=False
)
df_demo_24_25_lva.to_csv(
    "data/bva/csv/Registration_demographicdata_lva_24_25.csv", index=False
)

In [ ]:
len(df_demo_25["showref"].unique())

In [ ]:
len(df_demo_25["BadgeId"].unique())

In [ ]:
# List of Badges that we have registred in this year
demo_badge_id_25 = list(df_demo_25["BadgeId"].unique())
demo_badge_id_24_bva = list(df_demo_24["BadgeId"].unique())
demo_badge_id_24_lva = list(df3["BadgeId"].unique())

In [ ]:
demo_badge_id_24_bva[0]

# Registration data which have Demo Data this year , for Past year we are only interested in Demo data and reg data for Visitors in 2025 that came in 2024 to BVA or LVA

In [ ]:
df_reg_25_wdemo_data = df_reg_25[df_reg_25["BadgeId"].isin(demo_badge_id_25)]
df_reg_24_wdemo_data_bva = df_reg_24_25_bva[
    df_reg_24_25_bva["BadgeId"].isin(demo_badge_id_24_bva)
]
df_reg_24_wdemo_data_lva = df_reg_24_25_lva[
    df_reg_24_25_lva["BadgeId"].isin(demo_badge_id_24_lva)
]
len(df_reg_25_wdemo_data), len(df_reg_24_wdemo_data_bva), len(df_reg_24_wdemo_data_lva)

In [ ]:
def concatenate_qa_registration_data(df_registration, list_badge_id_events):
    """
    Concatenate Registration Values of a Particpant"
    Args:
        df_registration : Pandas Dataframe with values of Participants
        list_badge_id_events : List with all BadgeId from which we have Geographic Data. Answer to the questionarie
    """

    def create_string_from_dict(data):
        keys_to_exclude = ["ShowRef", "BadgeId"]
        result = []

        for key, value in data.items():
            if key not in keys_to_exclude:
                if value is not None:
                    result.append(f"{key}: {value}")
                else:
                    result.append(f"{key}: no_data")

        return ", ".join(result)

    reg_data_list = []
    for badge_ID, i in zip(list_badge_id_events, range(len(list_badge_id_events))):
        df = df_registration[df_registration["BadgeId"] == badge_ID]
        df_d = json.loads(df.to_json(orient="records"))
        reg_data = {}
        for row in df_d:
            ID = "_".join([row.get("ShowRef"), row.get("BadgeId")])
            reg_data[ID] = create_string_from_dict(row)
        if len(reg_data.keys()) > 0:
            reg_data_list.append(reg_data)

    return reg_data_list

In [ ]:
reg_data_list_this_year = concatenate_qa_registration_data(
    df_reg_25_valid_columns, demo_badge_id_25
)
reg_data_list_past_year_bva = concatenate_qa_registration_data(
    df_reg_24_25_bva_valid_columns, demo_badge_id_24_bva
)
reg_data_list_past_year_lva = concatenate_qa_registration_data(
    df_reg_24_25_lva_valid_columns, demo_badge_id_24_lva
)

In [ ]:
len(reg_data_list_this_year), len(reg_data_list_past_year_bva), len(
    reg_data_list_past_year_lva
)

In [ ]:
# Control Numbers
len(reg_data_list_this_year), len(demo_badge_id_25), len(
    reg_data_list_past_year_bva
), len(demo_badge_id_24_bva)

In [ ]:
reg_data_list_this_year[100], reg_data_list_past_year_bva[
    100
], reg_data_list_past_year_lva[100]

In [ ]:
# save Registration Data past and this year
with open("data/bva/output/dva_registration_data_this_year.json", "w") as f:
    json.dump(reg_data_list_this_year, f, indent=4)
with open("data/bva/output/dva_registration_data_past_year.json", "w") as f:
    json.dump(reg_data_list_past_year_bva, f, indent=4)
with open("data/bva/output/lva_registration_data_past_year.json", "w") as f:
    json.dump(reg_data_list_past_year_lva, f, indent=4)

# Choose which one Apply (Visitor with registration data and Demo Data)

In [ ]:
df_reg_25_wdemo_data["BadgeType"].unique()  # Choose which one Apply

In [ ]:
df_reg_24_wdemo_data_bva["BadgeType"].unique()

In [ ]:
df_reg_24_wdemo_data_lva["BadgeType"].unique()

In [ ]:
df_reg_25_wdemo_data.to_csv(
    "data/bva/output/Registration_data_with_demographicdata_bva_this.csv", index=False
)
df_reg_24_wdemo_data_bva.to_csv(
    "data/bva/output/Registration_data_with_demographicdata_bva_last.csv", index=False
)
df_reg_24_wdemo_data_lva.to_csv(
    "data/bva/output/Registration_data_with_demographicdata_lva_last.csv", index=False
)

# Concatenate Q&A Demografic Data

In [ ]:
# THIS BVA
{
    "Are you a",
    "Are you a BVA Member",
    "Are you a distributor to the profession",
    "Are you interested in exhibiting at the event",
    "Do you make purchases for your practice/organisation",
    "Free Product",
    "How do you plan to travel to the event",
    "How many nights do you intend to stay at a hotel for when attending this event",
    "I agree this event is aimed at registered veterinary professionals and by completing this registration I am agreeing that I am a registered veterinary professional If you arent a veterinary professional please purchase a commercial pass",
    "I have read and accept the Event Terms and Conditions",
    "I have read and accept the Privacy Policy",
    "If you are happy for your name and email address to be shared with such third parties for marketing purposes please tick here",
    "If you have received an education bursary your ticket can be cancelled by the 30th April if you do not cancel your pass and dont attend BVA Live you will be liable to pay a £99VAT non-attendance fee",
    "If you would like to receive information from us about third party products events and services please tick here",
    "Job Role",
    "Membership code",
    "Membership-checkboxes",
    "Organisation Type",
    "Referrer Name",
    "What can we do to make this event space more comfortable for you",
    "What type does your practice specialise in",
    "What veterinary school did you attend",
    "Which days will you be attending",
}
# PAST BVA

{
    "Are you a",
    "Are you a distributor to the profession",
    "Are you an employee of the exhibiting company",
    "Are you interested in exhibiting at the event",
    "Do you have an education bursary code",
    "Do you have any dietary requirements",
    "Have you attended a Vet Show previously",
    "How did you hear about us",
    "How do you plan to travel to the event",
    "How many nights do you intend to stay at a hotel for when attending this event",
    "I agree this event is aimed at registered veterinary professionals and by completing this registration I am agreeing that I am a registered veterinary professional If you arent a veterinary professional please purchase a commercial pass",
    "I have read and accept the Event Terms and Conditions",
    "I have read and accept the Privacy Policy",
    "If you are happy for your name and email address to be shared with such third parties for marketing purposes please tick here",
    "If you have received an education bursary your ticket can be cancelled by the 30th April if you do not cancel your pass and dont attend BVA Live you will be liable to pay a £99VAT non-attendance fee",
    "If you would like to receive information from us about third party products events and services please tick here",
    "Job Role",
    "Membership code",
    "Organisation Type",
    "Please enter you reference code here",
    "Referrer Name",
    "What areas do you specialise in",
    "What can we do to make this event space more comfortable for you",
    "What is your RCVS Number",
    "What veterinary school did you attend",
    "What year did you graduate",
}

# PAST LVA
{
    "Are you a",
    "Are you a distributor to the profession",
    "Are you interested in exhibiting at the event",
    "Do you have any dietary requirements",
    "Do you make purchases for your practice/organisation",
    "Have you attended another Vet Show",
    "How did you hear about the show",
    "How do you plan to travel to the event",
    "How many nights do you intend to stay at a hotel for when attending this event",
    "I agree this event is aimed at registered veterinary professionals and by completing this registration I am agreeing that I am a registered veterinary professional",
    "I have read and accept the Event Terms and Conditions",
    "I have read and accept the Privacy Policy",
    "If you are a BEVA Member what is your number",
    "If you are a BVA Member what is your number",
    "If you are happy for your name and email address to be shared with such third parties for marketing purposes please tick here",
    "If you would like to receive information from us about third party products events and services please tick here",
    "Job Role",
    "Organisation Type",
    "TRACESOURCECODE",
    "UTMCAMPAIGN",
    "UTMMEDIUM",
    "UTMSOURCE",
    "What areas do you specialise in",
    "What can we do to make this event space more comfortable for you",
    "What veterinary school did you attend",
    "What year did you graduate",
    "Which days will you be attending",
    "Your membership number",
}

In [ ]:
list_keep_this = [
    "What type does your practice specialise in",
    "Organisation Type",
    "Job Role",
]
list_keep_past = ["What areas do you specialise in", "Organisation Type", "Job Role"]

In [ ]:
new_columns_this = [x.lower().replace(" ", "_") for x in list_keep_this]
new_columns_last = [x.lower().replace(" ", "_") for x in list_keep_past]

In [ ]:
df_demo_25["AnswerText_backup"] = df_demo_25["AnswerText"].astype(str)
df_demo_24["AnswerText_backup"] = df_demo_24["AnswerText"].astype(str)
df3["AnswerText_backup"] = df3["AnswerText"].astype(str)

In [ ]:
org_type = df_demo_25[df_demo_25["QuestionText"] == "Organisation Type"]

In [ ]:
org_type["AnswerText"].unique()

In [ ]:
def concatenate_qa_demografic_data_pa(df_demografic, list_badge_id_events, list_keep):
    """
    Concatenate Questions and Answers of a Participant
    Args:
        df_demografic : Pandas Dataframe with values of Participants
        list_badge_id_events : List with all BadgeId from which we have Geographic Data. Answer to the questionnaire
        list_keep : List of questions that should be keept
    """
    demo_data = []

    for i, badge_ID in enumerate(list_badge_id_events):
        df = df_demografic[df_demografic["BadgeId"] == badge_ID]
        df_d = json.loads(df.to_json(orient="records"))
        qa = {badge_ID: {}}
        for row in df_d:

            question = row.get("QuestionText")
            answer = row.get("AnswerText")

            if not question or not answer:
                logging.warning(
                    f"Issue with records ID {badge_ID} NO ANSWER/QUESTION, row {i}, length dataframe: {len(df)}"
                )
                continue
            if question in list_keep:
                qq = question.lower().replace(" ", "_")
                qa[badge_ID][qq] = answer
        if len(qa[badge_ID].keys()) > 0:
            demo_data.append(qa)
        else:
            logging.warning(f"No demo data for {badge_ID}")
    return demo_data

In [ ]:
# demo_badge_id_25

In [ ]:
demo_data_this = concatenate_qa_demografic_data_pa(
    df_demo_25, demo_badge_id_25, list_keep_this
)

In [ ]:
demo_data_last_bva = concatenate_qa_demografic_data_pa(
    df_demo_24, demo_badge_id_24_bva, list_keep_past
)

In [ ]:
demo_data_last_lva = concatenate_qa_demografic_data_pa(
    df3, demo_badge_id_24_lva, list_keep_past
)

In [ ]:
len(demo_data_this), len(demo_data_last_bva), len(
    demo_data_last_lva
)  # 22042025 (1549, 1668, 5411)

In [ ]:
# BDAWL25_NPL234P  106

In [ ]:
demo_data_this[50], demo_data_last_bva[60], demo_data_last_lva[100]

In [ ]:
with open("data/bva/output/demographic_data_this.json", "w") as f:
    json.dump(demo_data_this, f, indent=4)
with open("data/bva/output/demographic_data_last_bva.json", "w") as f:
    json.dump(demo_data_last_bva, f, indent=4)
with open("data/bva/output/demographic_data_last_lva.json", "w") as f:
    json.dump(demo_data_last_lva, f, indent=4)

# Concatenate Scans to Demografic_data

In [ ]:
import pandas as pd
import json
from pandas import json_normalize

In [ ]:
def create_col_placeholders(df, list_to_keep):

    for col in list_to_keep:
        qq = col.lower().replace(" ", "_")
        df[qq] = "NA"
    return df

In [ ]:
df_reg_25_valid_columns = create_col_placeholders(
    df_reg_25_valid_columns, list_keep_this
)
df_reg_24_valid_columns = create_col_placeholders(
    df_reg_24_valid_columns, list_keep_past
)

In [ ]:
def create_democols_in_registration_data(df, demo_data, list_keep):
    """
    Populate columns demo data with its values
    """
    # create new columns
    df = create_col_placeholders(df, list_keep)
    # populate with demo data
    for reg in demo_data:
        badgeid = list(reg.keys())[0]
        list_keys_this_reg = list(reg[badgeid].keys())
        for col in list_keys_this_reg:
            df.loc[df["BadgeId"] == badgeid, col] = reg[badgeid][col]
    return df

In [ ]:
df_reg_demo_this = create_democols_in_registration_data(
    df_reg_25_valid_columns, demo_data_this, list_keep_this
)

In [ ]:
df_reg_demo_last_bva = create_democols_in_registration_data(
    df_reg_24_25_bva_valid_columns, demo_data_last_bva, list_keep_past
)

In [ ]:
df_reg_demo_last_lva = create_democols_in_registration_data(
    df_reg_24_25_lva_valid_columns, demo_data_last_lva, list_keep_past
)

In [ ]:
df_reg_demo_this.Days_since_registration.unique()

In [ ]:
df_reg_demo_this.head()

In [ ]:
df_reg_demo_last_bva.head(3)

In [ ]:
import re
from difflib import SequenceMatcher


def process_job_roles(df):
    # Make a copy to avoid modifying the original dataframe
    df_copy = df.copy()

    # Only process rows where job_role is "NA"
    mask = df_copy["job_role"] == "NA"

    # Define potential roles
    potential_roles = [
        "Student",
        "Other (please specify)",
        "Receptionist",
        "Head Nurse/Senior Nurse",
        "Vet/Vet Surgeon",
        "Practice Partner/Owner",
        "Academic",
        "Clinical or other Director",
        "Assistant Vet",
        "Vet/Owner",
        "Vet Nurse",
        "Locum Vet",
        "Practice Manager",
        "Locum RVN",
    ]

    # Apply each rule in sequence
    for idx in df_copy[mask].index:
        job_title = str(df_copy.loc[idx, "JobTitle"]).lower()

        # Rule 1-6: Check for specific strings in JobTitle
        if "surgeon" in job_title:
            df_copy.loc[idx, "job_role"] = "Vet/Vet Surgeon"
        elif "nurse" in job_title:
            df_copy.loc[idx, "job_role"] = "Vet Nurse"
        elif "rvn" in job_title:
            df_copy.loc[idx, "job_role"] = "Vet Nurse"
        elif "locum" in job_title:
            df_copy.loc[idx, "job_role"] = "Locum Vet"
        elif "student" in job_title:
            df_copy.loc[idx, "job_role"] = "Student"
        elif "assistant" in job_title:
            df_copy.loc[idx, "job_role"] = "Assistant Vet"
        else:
            # Rule 7: Use text similarity to find the best matching role

            # Clean the job title: remove common words that might interfere with matching
            cleaned_title = re.sub(r"\b(and|the|of|in|at|for)\b", "", job_title)

            # Find the role with highest similarity score
            best_match = None
            best_score = 0

            for role in potential_roles:
                # Calculate similarity between job title and each potential role
                role_lower = role.lower()

                # Check for key terms in the role
                role_terms = role_lower.split("/")
                role_terms.extend(role_lower.split())

                # Calculate max similarity with any term in the role
                max_term_score = 0
                for term in role_terms:
                    if len(term) > 2:  # Only consider meaningful terms
                        if term in cleaned_title:
                            term_score = 0.9  # High score for direct matches
                        else:
                            # Use sequence matcher for fuzzy matching
                            term_score = SequenceMatcher(
                                None, cleaned_title, term
                            ).ratio()
                        max_term_score = max(max_term_score, term_score)

                if max_term_score > best_score:
                    best_score = max_term_score
                    best_match = role

            # If similarity is above threshold, use the best match
            if best_score > 0.3:  # Adjustable threshold
                df_copy.loc[idx, "job_role"] = best_match
            else:
                # Default to "Other" if no good match
                df_copy.loc[idx, "job_role"] = "Other (please specify)"

    # Final rule: Replace any occurrence of "Other" with "Other (please specify)"
    other_mask = df_copy["job_role"].str.contains(
        "Other", case=False, na=False
    ) & ~df_copy["job_role"].eq("Other (please specify)")
    df_copy.loc[other_mask, "job_role"] = "Other (please specify)"

    return df_copy

In [ ]:
df_reg_demo_this["job_role"].value_counts()

In [ ]:
df_reg_demo_this = process_job_roles(df_reg_demo_this)
df_reg_demo_this["job_role"].value_counts()

In [ ]:
df_reg_demo_last_bva = process_job_roles(df_reg_demo_last_bva)
df_reg_demo_last_bva["job_role"].value_counts()

In [ ]:
df_reg_demo_last_lva = process_job_roles(df_reg_demo_last_lva)
df_reg_demo_last_lva["job_role"].value_counts()

In [ ]:
df_reg_demo_this.to_csv("data/bva/output/df_reg_demo_this.csv", index=False)
df_reg_demo_last_bva.to_csv("data/bva/output/df_reg_demo_last_bva.csv", index=False)
df_reg_demo_last_lva.to_csv("data/bva/output/df_reg_demo_last_lva.csv", index=False)

In [ ]:
df_reg_demo_this["what_type_does_your_practice_specialise_in"].value_counts()

In [ ]:
df_reg_demo_last_lva["what_areas_do_you_specialise_in"].value_counts()

In [ ]:
df_reg_demo_last_bva["what_areas_do_you_specialise_in"].value_counts()

In [ ]:
practices = pd.read_csv("data/bva/practices_missing.csv")
practices.head()

In [ ]:
practices.columns

In [ ]:
df_reg_demo_this.columns

In [ ]:
! pip install fuzzywuzzy 
 


In [ ]:
import pandas as pd
from fuzzywuzzy import process


def fill_missing_practice_types(
    df_reg_demo_this, practices, column="what_type_does_your_practice_specialise_in"
):
    """
    Fill practice types in df_reg_demo_this that have value "NA" by
    fuzzy matching company names with the practices dataframe.

    Parameters:
    -----------
    df_reg_demo_this : pandas.DataFrame
        Dataframe containing registration data with "NA" practice types
    practices : pandas.DataFrame
        Dataframe containing company names and their practice types

    Returns:
    --------
    pandas.DataFrame
        Updated dataframe with filled practice types
    """
    # Create a copy of the input dataframe to avoid modifying the original
    df = df_reg_demo_this.copy()

    # Identify rows where practice types are "NA"
    missing_idx = df[column] == "NA"
    companies_to_match = df.loc[missing_idx, "Company"].tolist()

    # Create a dictionary of company names from practices dataframe
    practice_types_dict = dict(
        zip(practices["Company Name"], practices["Main Type of Veterinary Practice"])
    )

    # List of all company names in practices dataframe for fuzzy matching
    all_practice_companies = practices["Company Name"].tolist()

    # For each company with "NA" practice type, find the best match
    for idx, company in zip(df.loc[missing_idx].index, companies_to_match):
        # Skip if company name is missing or empty
        if pd.isna(company) or company == "":
            continue

        # Find the best match using fuzzy matching
        best_match, score = process.extractOne(company, all_practice_companies)

        # Only update if match score is good enough (adjust threshold as needed)
        if score >= 95:  # 95% match threshold, adjust as needed
            df.loc[idx, column] = practice_types_dict[best_match]
            print(
                f"Matched '{company}' to '{best_match}' with score {score}. New type: {practice_types_dict[best_match]}"
            )

    return df


# Example usage:
# result_df = fill_missing_practice_types(df_reg_demo_this, practices)

In [ ]:
df_reg_demo_this["what_type_does_your_practice_specialise_in"].value_counts()

In [ ]:
df_reg_demo_this_2 = fill_missing_practice_types(
    df_reg_demo_this, practices, column="what_type_does_your_practice_specialise_in"
)

In [ ]:
df_reg_demo_this_2["what_type_does_your_practice_specialise_in"].value_counts()

In [ ]:
df_reg_demo_last_lva["what_areas_do_you_specialise_in"].value_counts()

In [ ]:
df_reg_demo_last_lva_2 = fill_missing_practice_types(
    df_reg_demo_last_lva, practices, column="what_areas_do_you_specialise_in"
)

In [ ]:
df_reg_demo_last_lva_2["what_areas_do_you_specialise_in"].value_counts()

In [ ]:
df_reg_demo_last_bva["what_areas_do_you_specialise_in"].value_counts()

In [ ]:
df_reg_demo_last_bva_2 = fill_missing_practice_types(
    df_reg_demo_last_bva, practices, column="what_areas_do_you_specialise_in"
)

In [ ]:
df_reg_demo_last_bva_2["what_areas_do_you_specialise_in"].value_counts()

In [ ]:
df_reg_demo_this_2.to_csv("data/bva/output/df_reg_demo_this.csv", index=False)
df_reg_demo_last_bva_2.to_csv("data/bva/output/df_reg_demo_last_bva.csv", index=False)
df_reg_demo_last_lva_2.to_csv("data/bva/output/df_reg_demo_last_lva.csv", index=False)

In [ ]:
len(df_reg_demo_this_2), len(df_reg_demo_last_bva_2), len(df_reg_demo_last_lva_2)

In [ ]:
df_reg_demo_this_2.columns

In [ ]:
len(df_reg_demo_this_2["BadgeId"].unique())

In [ ]:
df_reg_demo_this_2.isnull().sum()

In [ ]:
df_reg_demo_this_2["job_role"].value_counts()